In [605]:
a = [1, 2, 3, 4, 5, 6]
a[-4:]

[3, 4, 5, 6]

In [606]:
import numpy as np


class PhasorAverageModel:
	horizon = 10000
	Likelihood = [0] * horizon
	Coherence = [0] * horizon
	waves = np.empty((0, 3), dtype=float)
	likelihood = 0
	angular_freq = np.nan
	buffer = [None]
	phase_angle = 0
	amplitudeFunc = None
	envelopeFunc = None
	omegaFunc = None
	model_statistics = None
	last_event = 0

	@staticmethod
	def processWaves(waves):
		return waves[2:]

	@staticmethod
	def phasorValues(waves, last_event=0):
		omega = waves[:, 0][-last_event:]
		T = waves[:, 1][-last_event:]	# phase angle defined from common start referene t=0
		amplitude = np.ones_like(T) if waves.shape[-1] == 2 else waves[:, 2][-last_event:]
		t = T[-1]

		phi = t - T
		return omega, phi, amplitude

	@staticmethod
	def calculateHorizon(horizon, phi):
		if phi.shape[0] > 2:
			return horizon
		if phi.shape[0] > 6:
			d = np.diff(phi)
			statistic = max(d)
		else:
			statistic = np.mean(phi)

		if 3 * statistic > horizon:
			return int(2 * statistic)	# just to be safe
		return horizon

	@staticmethod
	def getPhasors(amplitude, envelope, omega):
		return amplitude * np.exp(-envelope) * np.exp(1j * omega)

	@staticmethod
	def phasorStatistics(phasor, last_event=0):

		s_sum = np.sum(phasor, axis=1)
		m_sum = np.abs(phasor).sum(axis=1)
		psi = s_sum / m_sum
		coherence = np.abs(psi)
		return psi, coherence

	@staticmethod
	def getLikelihood(psi, coherence, mode="normalised", func=None, **kwargs):
		if mode == "normalised":
			return (coherence + psi.real) / 2
		if mode == "max":
			return np.maximum(0, psi.real)
		if mode == "idk":
			return np.abs(psi)
		if func is not None:
			return func(psi, coherence, **kwargss)
		raise ValueError(f"{mode} is not defined")

	@staticmethod
	def predictPhasorAverage(
		waves,
		*,
		horizon=2000,
		# amplitudeFunc=None,
		# envelopeFunc=None,
		# omegaFunc=None,
		last_event=0,
		**kwargs,
	):
		"""
		waves=[(angular_freq,phase_angle_k,amplitude),...]
		"""
		waves = PhasorAverageModel.processWaves(
			waves,
		)

		if len(waves) == 0:
			return np.zeros(horizon), np.zeros(horizon)

		omega, phi, amplitude = PhasorAverageModel.phasorValues(waves, last_event)
		horizon = PhasorAverageModel.calculateHorizon(horizon, phi)
		# horizon = len(waves) - last_event
		Delta_t = np.arange(0, horizon + 1)[:, None]

		def amplitudeFunc(amplitude, **kwargs):
			return amplitude	# np.ones_like(amplitude)

		def envelopeFunc(phi, **kwargs):
			return np.zeros_like(phi)

		def omegaFunc(omega, phi, **kwargs):
			return phi * omega

		Envelope_t = envelopeFunc(phi)
		Omega_t = omegaFunc(omega, phi)
		Amplitude_t = amplitudeFunc(amplitude)

		Envelope_rot = envelopeFunc(Delta_t, **kwargs)
		Omega_rot = omegaFunc(omega, Delta_t, **kwargs)
		Amplitude_rot = np.ones_like(Delta_t, **kwargs)

		x_now = PhasorAverageModel.getPhasors(Amplitude_t, Envelope_t, Omega_t)
		x_rot = PhasorAverageModel.getPhasors(Amplitude_rot, Envelope_rot, Omega_rot)
		x_project = x_rot * x_now

		psi, coherence = PhasorAverageModel.phasorStatistics(x_project, last_event)
		likelihood = PhasorAverageModel.getLikelihood(**locals())

		return [likelihood, coherence]

	@staticmethod
	def construct_wave(angular_freq, phase_angle, amplitude):
		return np.array((angular_freq, phase_angle, amplitude))

	@staticmethod
	def valid_angular_frequency(waves):
		return len(waves) > 1

	@staticmethod
	def update_angular_frequency(x, phase_angle):

		return 2 * np.pi / (x - phase_angle)

	@staticmethod
	def update_angular_phase_angle(x, phase_angle):

		return x - phase_angle

	@staticmethod
	def get_current_likelihood(x, Likelihood, phase_angle):
		return Likelihood[x - phase_angle]

	@staticmethod
	def get_current_coherence(x, Coherence, phase_angle):

		return Coherence[x - phase_angle]

	@staticmethod
	def construct_waves(wave, waves):
		return np.concat([waves, [wave]])

	def __init__(self):
		pass

	def __call__(self, x):
		if self.valid_angular_frequency(self.waves):
			self.angular_freq = self.update_angular_frequency(x, self.phase_angle)

		time_delta = int(x - self.phase_angle)
		current_likelihood = self.Likelihood[time_delta]
		current_coherence = self.Coherence[time_delta]

		return current_likelihood, current_coherence

	def event(self, x, trigger):

		self.phase_angle = x
		amplitude = trigger

		wave = self.construct_wave(self.angular_freq, self.phase_angle, amplitude)
		self.waves = self.construct_waves(wave, self.waves)

		self.model_statistics = self.predictPhasorAverage(
			waves=self.waves,
			horizon=self.horizon,
			# amplitudeFunc=self.amplitudeFunc,
			# envelopeFunc=self.envelopeFunc,
			# omegaFunc=self.omegaFunc,
		)
		self.last_event = x

		self.Likelihood = self.model_statistics[0]
		self.Coherence = self.model_statistics[1]

		return self.Likelihood, self.Coherence

In [607]:
# def getRegimeIndices(n_events, periods, start_at_origin=False, **kwargs):
# 	sub_signals = np.vectorize(np.arange, otypes=[object])(n_events) * periods
# 	last_index = np.array([0, *[item[-1] for item in sub_signals[:-1]]])
# 	period_off = np.cumsum(periods) - periods[0] * int(start_at_origin)
# 	offset = period_off + np.cumsum(last_index)
# 	return np.concat((sub_signals + np.array(offset)))


# def getRealSignal(regimes, start_at_origin=False, **kwargs):
# 	n_events, periods = np.array(regimes)[:, :2].T
# 	indices = getRegimeIndices(n_events, periods, **kwargs)
# 	true_signal = np.zeros((n_events @ periods) + 1)
# 	true_signal[indices] = 1
# 	return true_signal


# SIGNAL_REGIMES = [
# 	(25, 30),	# 25 events with period 30
# 	(20, 70),	# 20 events with period 70
# 	(30, 45),	# 30 events with period 45
# ]
# true_signal = getRealSignal(SIGNAL_REGIMES, start_at_origin=True)

# sig_len = len(true_signal)

# timestamps = np.arange(sig_len)

# real_pulse_times = timestamps[true_signal == 1]


# def generate_truncated_cauchy(n_samples, location=0.0, scale=0.001, low=-1.0, high=1.0):

# 	def cdf(x):
# 		return 0.5 + (1 / np.pi) * np.arctan((x - location) / scale)

# 	def inverse_cdf(p):
# 		return location + scale * np.tan(np.pi * (p - 0.5))

# 	cdf_low = cdf(low)
# 	cdf_high = cdf(high)

# 	uniform_samples = np.random.uniform(0.0, 1.0, n_samples)

# 	prob_samples = cdf_low + uniform_samples * (cdf_high - cdf_low)

# 	final_samples = inverse_cdf(prob_samples)

# 	return final_samples


# noise = generate_truncated_cauchy(sig_len)


# m = PhasorAverageModel()
# for x, y in enumerate(true_signal):
# 	m(x)
# 	if y == 1:
# 		m.event(x, 1 + (noise[x]))

# # h = len(m.Likelihood)
# # print(m.Likelihood)


# # sig_len = len(true_signal)
# # h = np.arange(len(m.Likelihood)) + sig_len - 1	# - SIGNAL_REGIMES[0][-1]
# # fig = go.Figure()
# # for t in real_pulse_times:
# # 	fig.add_shape(
# # 		type="line",
# # 		x0=t,
# # 		y0=0,
# # 		x1=t,
# # 		y1=1 + (noise[t]),
# # 		line=dict(color="rgba(0, 0, 0, 0.5)", width=1.5),
# # 	)
# # fig = go.Figure()

# # fig.add_trace(
# # 	go.Scatter(
# # 		x=h,
# # 		y=(m.Likelihood),
# # 		mode="lines",
# # 		# mode="markers",
# # 		name="Likelihoods",
# # 		# line=dict(color="rgb(255, 0, 0, 0.5)", width=4),
# # 	)
# # )
# # fig.add_trace(
# # 	go.Scatter(
# # 		x=h,
# # 		y=m.Coherence,
# # 		mode="lines",
# # 		# mode="markers",
# # 		name="Coherence",
# # 		# line=dict(color="rgb(255, 0, 0, 0.5)", width=4),
# # 	)
# # )
# # fig.show()

In [608]:
from scipy.stats import gennorm


def generate_truncated_gennorm(
	n_samples, beta=1, alpha=0.7, location=0.0, low=-1.0, high=1.0
):

	cdf_low = gennorm.cdf(low, beta, loc=location, scale=alpha)
	cdf_high = gennorm.cdf(high, beta, loc=location, scale=alpha)

	uniform_samples = np.random.uniform(0.0, 1.0, n_samples)

	prob_samples = cdf_low + uniform_samples * (cdf_high - cdf_low)

	final_samples = gennorm.ppf(prob_samples, beta, loc=location, scale=alpha)

	return final_samples

In [609]:
import numpy as np
import matplotlib.pyplot as plt


def generate_truncated_cauchy(n_samples, location=0.0, scale=0.05, low=-1.0, high=1.0):

	def cdf(x):
		return 0.5 + (1 / np.pi) * np.arctan((x - location) / scale)

	def inverse_cdf(p):
		return location + scale * np.tan(np.pi * (p - 0.5))

	cdf_low = cdf(low)
	cdf_high = cdf(high)

	uniform_samples = np.random.uniform(0.0, 1.0, n_samples)

	prob_samples = cdf_low + uniform_samples * (cdf_high - cdf_low)

	final_samples = inverse_cdf(prob_samples)

	return final_samples

In [610]:
def getRegimeIndices(n_events, periods, start_at_origin=False, **kwargs):
	# array_r = np.array(regimes)
	# n_events, periods = array_r[:, :2].T
	# total_time = n_events @ periods
	sub_signals = np.vectorize(np.arange, otypes=[object])(n_events) * periods
	last_index = np.array([0, *[item[-1] for item in sub_signals[:-1]]])
	period_off = np.cumsum(periods) - periods[0] * int(start_at_origin)
	offset = period_off + np.cumsum(last_index)
	return np.concat((sub_signals + np.array(offset)))


def getRealSignal(regimes, **kwargs):
	array_r = np.array(regimes)
	n_events, periods = array_r[:, :2].T
	total_time = n_events @ periods
	true_signal = np.zeros(total_time)
	indices = getRegimeIndices(n_events, periods, **kwargs)
	true_signal[indices] = 1
	print(true_signal)
	pass


SIGNAL_REGIMES = [
	(25, 30),	# 25 events with period 30
	(20, 70),	# 20 events with period 70
	(30, 45),	# 30 events with period 45
]

getRealSignal(SIGNAL_REGIMES, start_at_origin=True)

# np.arange(25) * 3

[1. 0. 0. ... 0. 0. 0.]


In [611]:
a = np.zeros(10, dtype=int)
idx = np.array([2, 4, 7])

a[idx] = 1
a

array([0, 0, 1, 0, 1, 0, 0, 1, 0, 0])

In [612]:
def pulseDector(difference, armed=[False], d_up=6.0):	# d_up is threshold
	# mutable parameter holds state between invocations, ensures we dont have sequential pulses

	fire = False

	if armed[0]:
		if (
			difference < 0
		):	# its valid to fire after any decrease in the diff, that is, it is only disarmed when the function is strictly increasing

			armed[0] = False
	else:

		if difference >= d_up:
			fire = True
			armed[0] = True
	return fire


def pulseDector2(difference, armed=[False], d_up=6.0):	# d_up is threshold
	# mutable parameter holds state between invocations, ensures we dont have sequential pulses

	fire = False

	if armed[0]:
		if (
			difference < 0
		):	# its valid to fire after any decrease in the diff, that is, it is only disarmed when the function is strictly increasing

			armed[0] = False
	else:

		if difference >= d_up:
			fire = True
			armed[0] = True
	return fire

In [613]:
data = np.load("sensordata.npz")

D_UP = 3

x = data["x"]
y = data["y"]


class events:
	def update(self, t, d, l, c, *, fire=False):
		self.times.append(t)
		self.differences.append(d)
		self.likelihoods.append(l)
		self.coherences.append(c)
		if fire:
			self.firings.append(t)

	def __init__(self):
		self.firings = []
		self.differences = []
		self.likelihoods = []
		self.coherences = []
		self.times = []


model = PhasorAverageModel()
base_track = events()
mod_track = events()
breakpoint_ = 10


def update(fig, x, y):
	fig.data[0].y = y
	fig.data[0].x = x
	display(fig)
	return fig


for t, sample in enumerate(y):
	l, c = model(t)
	# diff = y[t] - y[t - 2] if t > 1 else 0
	diff = (y[t] - y[t - 2]) / 2 if t > 1 else 0

	fire_base = pulseDector(diff, d_up=D_UP)

	base_track.update(t, diff, 0, 0, fire=fire_base)

	diff_2 = diff + (diff * ((l) + (1 - c)))
	fire_mod = pulseDector2(diff_2, d_up=D_UP)
	if fire_mod:
		L, U = model.event(t, diff)

		# sfig = update(sfig, np.arange(len(L[:500])) + t, L[:500])

	mod_track.update(t, diff_2, l, (1 - c), fire=fire_mod)

In [614]:
data = np.load("sensordata.npz")

D_UP = 3

x = data["x"]
y = data["y"]


class events:
	def update(self, t, d, l, c, *, fire=False, Likelihoods=None, Coherences=None):
		self.times.append(t)
		self.differences.append(d)
		self.likelihoods.append(l)
		self.coherences.append(c)
		if fire:
			self.firings.append(t)
		if Likelihoods is not None:
			self.Likelihoods.append(Likelihoods)
		if Coherences is not None:
			self.Coherences.append(Coherences)

	def __init__(self):
		self.firings = []
		self.differences = []
		self.likelihoods = []
		self.coherences = []
		self.Likelihoods = []
		self.Coherences = []

		self.times = []


model = PhasorAverageModel()
base_track = events()
mod_track = events()
breakpoint_ = 10


def pulseDector(difference, armed=[False], d_up=6.0):	# d_up is threshold
	# mutable parameter holds state between invocations, ensures we dont have sequential pulses

	fire = False

	if armed[0]:
		if (
			difference < 0
		):	# its valid to fire after any decrease in the diff, that is, it is only disarmed when the function is strictly increasing

			armed[0] = False
	else:

		if difference >= d_up:
			fire = True
			armed[0] = True
	return fire


L = []
C = []
has_base = False
for t, sample in enumerate(y):
	# l, c = model(t)
	# diff = (y[t] - y[t - 1]) if t > 0 else 0

	# fire_base = pulseDector(diff, d_up=2)

	# base_track.update(t, diff, 0, 0, fire=fire_base)

	# diff_2 = diff + (diff * (l + (1 + -c)))
	# fire_mod = pulseDector2(diff_2, d_up=2.2)
	l, c = model(t)
	# diff = y[t] - y[t - 1] if t > 0 else 0
	diff = (y[t] - y[t - 2])/2 if t > 1 else 0
	fire_base = pulseDector(diff, d_up=1.5)

	base_track.update(t, diff, 0, 0, fire=fire_base)

	diff_2 = diff + (diff * ((l)))

	if fire_base:
		has_base = True

	if has_base:
		fire_mod = pulseDector2(diff_2, d_up=1.5 - c)
	else:
		fire_mod = False

	if fire_mod:
		L, C = model.event(t, diff)
	mod_track.update(t, diff_2, l, c, fire=fire_mod, Likelihoods=L, Coherences=C)

In [615]:
from plotly.subplots import make_subplots

import plotly.graph_objects as go

fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1)
fig.add_trace(
	go.Scatter(
		x=list(range(t)), y=y, mode="lines", name="Signal", line=dict(color="blue")
	),
	row=1,
	col=1,
)
x_lines1 = []
y_lines1 = []
y_min = 0
y_max = max(y)	# just to span entire height
for f in base_track.firings:
	x_lines1.append(f)
	y_lines1.append(y_min)
	x_lines1.append(f)
	y_lines1.append(y_max)
	x_lines1.append(np.nan)
	y_lines1.append(np.nan)

fig.add_trace(
	go.Scatter(
		x=x_lines1,
		y=y_lines1,
		# mode="lines",
		name="Original Detected Pulse",
		line=dict(color="black", width=2, dash="dash"),
		showlegend=True,
	),
	row=1,
	col=1,
)
x_lines2 = []
y_lines2 = []

for g in mod_track.firings:
	x_lines2.append(g)
	y_lines2.append(y_min)
	x_lines2.append(g)
	y_lines2.append(y_max)
	x_lines2.append(np.nan)
	y_lines2.append(np.nan)

fig.add_trace(
	go.Scatter(
		x=x_lines2,
		y=y_lines2,
		mode="lines",
		name="Modulated Detected Pulse",
		line=dict(color="red", width=1),
		showlegend=True,
	),
	row=1,
	col=1,
)
fig.add_trace(
	go.Scatter(
		x=list(range(t)),
		y=base_track.differences,
		name="Original Differences",
		line=dict(color="rgba(0,0,0,0.5)", width=2),
	),
	row=2,
	col=1,
)

fig.add_trace(
	go.Scatter(
		x=list(range(t)),
		y=mod_track.differences,
		name="Modulated Differences",
		line=dict(color="rgba(255,0,0,0.5)", width=1),
	),
	row=2,
	col=1,
)
fig.add_trace(
	go.Scatter(
		x=list(range(t)),
		y=(np.array(mod_track.likelihoods)),
		name="Likelihoods",
	),
	row=3,
	col=1,
)
fig.add_trace(
	go.Scatter(
		x=list(range(t)),
		y=np.array(mod_track.coherences),
		name="Coherences",
	),
	row=3,
	col=1,
)
fig.update_layout(width=1900, height=1000)

In [616]:
index = 963

fig = go.Figure()

fig.add_trace(
	go.Scatter(
		x=np.arange(len(mod_track.Likelihoods[index])) + index,
		y=mod_track.Likelihoods[index],
		name=f"Likelihoods at index {index}",
	)
)
fig.add_trace(
	go.Scatter(
		x=np.arange(len(mod_track.Coherences[index])) + index,
		y=mod_track.Coherences[index],
		name=f"Coherences at index {index}",
	)
)
fig.update_layout(width=1900)

In [617]:
# D_UP = 3

# x = data["x"]
# y = data["y"]


# class events:
# 	# firings = []
# 	# differences = []
# 	# likelihoods = []
# 	# coherences = []
# 	# times = []

# 	def update(self, t, d, l, c, *, fire=False):
# 		self.times.append(t)
# 		self.differences.append(d)
# 		self.likelihoods.append(l)
# 		self.coherences.append(c)
# 		if fire:
# 			self.firings.append(t)

# 	def __init__(self):
# 		self.firings = []
# 		self.differences = []
# 		self.likelihoods = []
# 		self.coherences = []
# 		self.times = []


# model = PhasorAverageModel()
# base_track = events()
# mod_track = events()
# breakpoint_ = 10
# first_detect = False
# sfig = go.Figure()
# # sfig.batch_update
# # display(sfig)
# sfig.add_trace(
# 	go.Scatter(
# 		x=[None],
# 		y=[None],
# 		mode="lines",
# 		name="Likelihoods",
# 	)
# )
# # sfig.add_trace(
# # 	go.Scatter(
# # 		x=[None],
# # 		y=[None],
# # 		mode="lines",
# # 		name="Coherence",
# # 	)
# # )
# sfig.show()


# def update(fig, x, y):
# 	fig.data[0].y = y
# 	fig.data[0].x = x
# 	display(fig)
# 	return fig


# for t, sample in enumerate(y):
# 	l, c = model(t)
# 	diff = y[t] - y[t - 1] if t > 0 else 0
# 	fire_base = pulseDector(diff, d_up=D_UP)

# 	base_track.update(t, diff, 0, 0, fire=fire_base)
# 	# if diff > 0:
# 	# 	diff_2 = diff + (diff * ((l) + (1 - c)))
# 	# else:
# 	# 	diff_2 = diff
# 	diff_2 = diff + (diff * ((l) + (1 - c)))
# 	fire_mod = pulseDector2(diff_2, d_up=D_UP)
# 	if fire_mod:
# 		L, U = model.event(t, diff_2)
# 		# with sfig.batch_update():
# 		# sfig.data = []
# 		sfig = update(sfig, np.arange(len(L[:500])) + t, L[:500])
# 		# sfig.data[0].y = L[:500]
# 		# sfig.data[0].x = np.arange(len(L[:500])) + t
# 		# display(sfig)
# 		# sfig.add_trace(
# 		# 	go.Scatter(
# 		# 		x=np.arange(len(L[:500])) + t,
# 		# 		y=(L[:500]),
# 		# 		mode="lines",
# 		# 		name="Likelihoods",
# 		# 	)
# 		# )
# 		# sfig.add_trace(
# 		# 	go.Scatter(
# 		# 		x=np.arange(len(L[:500])) + t,
# 		# 		y=U[:500],
# 		# 		mode="lines",
# 		# 		name="Coherence",
# 		# 	)
# 		# )
# 		# sfig.show()

# 	mod_track.update(t, diff_2, l, (1 - c), fire=fire_mod)

In [618]:
# from plotly.subplots import make_subplots

# from plotly.subplots import make_subplots

# fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1)
# fig.add_trace(
# 	go.Scatter(
# 		x=list(range(t)), y=y, mode="lines", name="Signal", line=dict(color="blue")
# 	),
# 	row=1,
# 	col=1,
# )
# x_lines1 = []
# y_lines1 = []
# y_min = 0
# y_max = max(y)	# just to span entire height
# for f in base_track.firings:
# 	x_lines1.append(f)
# 	y_lines1.append(y_min)
# 	x_lines1.append(f)
# 	y_lines1.append(y_max)
# 	x_lines1.append(np.nan)
# 	y_lines1.append(np.nan)

# fig.add_trace(
# 	go.Scatter(
# 		x=x_lines1,
# 		y=y_lines1,
# 		mode="lines",
# 		name="Original Detected Pulse",
# 		line=dict(color="black", width=2, dash="dash"),
# 		showlegend=True,
# 	),
# 	row=1,
# 	col=1,
# )
# x_lines2 = []
# y_lines2 = []

# for g in mod_track.firings:
# 	x_lines2.append(g)
# 	y_lines2.append(y_min)
# 	x_lines2.append(g)
# 	y_lines2.append(y_max)
# 	x_lines2.append(np.nan)
# 	y_lines2.append(np.nan)

# fig.add_trace(
# 	go.Scatter(
# 		x=x_lines2,
# 		y=y_lines2,
# 		mode="lines",
# 		name="Modulated Detected Pulse",
# 		line=dict(color="red", width=1),
# 		showlegend=True,
# 	),
# 	row=1,
# 	col=1,
# )

# fig.add_trace(
# 	go.Scatter(
# 		x=list(range(t)),
# 		y=mod_track.likelihoods,
# 		name="Likelihoods",
# 	),
# 	row=2,
# 	col=1,
# )
# fig.add_trace(
# 	go.Scatter(
# 		x=list(range(t)),
# 		y=mod_track.coherences,
# 		name="Coherences",
# 	),
# 	row=2,
# 	col=1,
# )

In [619]:
def pulseDector(difference, armed=[False], d_up=6.0):	# d_up is threshold
	# mutable parameter holds state between invocations, ensures we dont have sequential pulses

	fire = False

	if armed[0]:
		if (
			difference < 0
		):	# its valid to fire after any decrease in the diff, that is, it is only disarmed when the function is strictly increasing

			armed[0] = False
	else:

		if difference >= d_up:
			fire = True
			armed[0] = True
	return fire